In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 250)

import warnings
warnings.filterwarnings("ignore")

In [2]:
non_feature_cols = ['site','timestamp','demand_response','demand_response_capacity','date','busday','time','minute']
base_features = ['temp','irr','power']
target_cls = 'demand_response'
target_reg = 'demand_response_capacity'
working_hours = [10,11,12,13,14,15,16,17]

In [3]:
def conform(data):
    df = data.copy()
    working_hours = [10,11,12,13,14,15,16,17]

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['busday'] = np.is_busday(df['timestamp'].to_numpy().astype("datetime64[D]")).astype(int)
    df['date'] = pd.to_datetime(df['timestamp'].dt.date)
    df['day_of_week'] = df['date'].dt.weekday
    df['week'] = df['timestamp'].dt.isocalendar().week.astype(int)
    df['hour'] = df['timestamp'].dt.hour
    df['time'] = df['timestamp'].dt.time
    df['quarter_hour'] = df['timestamp'].dt.minute//15
    df['month'] = df['timestamp'].dt.month
    df['year'] = df['timestamp'].dt.year
    df['season'] = 0
    df.loc[df['month'].isin([12,1,2]), 'season'] = 0
    df.loc[df['month'].isin([3,4,5]), 'season'] = 1
    df.loc[df['month'].isin([6,7,8]), 'season'] = 2
    df.loc[df['month'].isin([9,10,11]), 'season'] = 3
    df['working_hours'] = 0
    df.loc[df['hour'].isin(working_hours), 'working_hours'] = 1

    return df

# Load prev train/test sites

In [4]:
train = pd.read_csv('./data/sites_ABC.csv')
train.columns = ['site','timestamp','temp','irr','power','demand_response','demand_response_capacity']
prev_test = pd.read_csv('./data/sites_DEF.csv')
prev_test.columns = ['site','timestamp','temp','irr','power','demand_response']
prev_test['demand_response_capacity'] = 0
prev_test = prev_test[prev_test['site'] == 'siteD']
prev_df = pd.concat([train, prev_test]).reset_index(drop=True)
prev_df = conform(prev_df)

# Load test sites

In [5]:
df = pd.read_csv('./data/test_sites_WithDemandResponsePred.csv', index_col=0)[['site','timestamp','temp','irr','power','demand_response']]
df['demand_response_capacity'] = 0
df = conform(df)

# Feature engineering

In [6]:
def interpolate_prediction(group):
    dr_nonzero = group['li_flag'] != 0
    nonzero_indices = group.index[dr_nonzero]

    if nonzero_indices.empty:
        group['li_feature'] = np.nan
        return group

    start_idx = nonzero_indices[0]
    end_idx = nonzero_indices[-1]

    before_idx = start_idx - 1
    after_idx = end_idx + 1

    if before_idx < group.index[0] or after_idx > group.index[-1]:
        group['li_feature'] = np.nan
        return group

    start_power = group.loc[before_idx, 'power']
    end_power = group.loc[after_idx, 'power']

    num_points = end_idx - start_idx + 1
    interpolated = np.linspace(start_power, end_power, num_points)

    group['li_feature'] = np.nan
    group.loc[start_idx:end_idx, 'li_feature'] = interpolated

    return group

def fill_between_activity(df):
    df = df.copy()

    def fill_day(g):
        # find first and last hour where pow_above_base == 1
        if g['pow_above_base'].sum() == 0:
            return g  # no active hours, do nothing
        first_on = g.loc[g['pow_above_base'] == 1, 'time'].min()
        last_on  = g.loc[g['pow_above_base'] == 1, 'time'].max()

        # fill all hours between those two with 1
        g.loc[(g['time'] >= first_on) & (g['time'] <= last_on), 'pow_above_base'] = 1
        return g

    df = df.groupby(['site', 'date'], group_keys=False).apply(fill_day)
    return df

def feature_engineering(og_df):
    df = og_df.copy()
    
    baseline_partition_columns = ['site', 'date']
    baseline_hours = [5,6,7]
    baseline_powers = df[df['hour'].isin(baseline_hours)].groupby(baseline_partition_columns)['power'].median().to_dict()
    df['baseline_pow'] = (
        df[baseline_partition_columns]
        .apply(tuple, axis=1)
        .map(baseline_powers)
        * 1.2
    )
    df['pow_above_base'] = (df['power'] > df['baseline_pow']).astype(int)
    df = fill_between_activity(df)
    df['site_on'] = df.groupby(['site','date'])['pow_above_base'].cumsum()
    df.loc[df['pow_above_base'] == 0, 'site_on'] = 0
    
    import datetime as dt
    df['li_flag'] = df['time'].apply(lambda t: int(dt.time(10,0) <= t <= dt.time(18,0)))
    df = df.groupby(['site', 'date'], group_keys=False).apply(interpolate_prediction)

    # df = df[df['working_hours'] == 1].reset_index(drop=True).copy()
    for var in ['temp', 'irr']:
        # per-site, month, time correlation of power with var
        corr_map = df.groupby(['site','month','day_of_week','time','year']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_mt')

        df = df.merge(corr_map, on=['site','month','day_of_week','time','year'], how='left')

        corr_map = df.groupby(['site','season','day_of_week','time','year']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_st')

        df = df.merge(corr_map, on=['site','season','day_of_week','time','year'], how='left')

        # per-site, month, time correlation of power with var POW ABOVE BASE ONLY
        corr_map = df.loc[df['pow_above_base'] == 1].groupby(['site','month','day_of_week','time','year']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_mt_pab')

        df = df.merge(corr_map, on=['site','month','day_of_week','time','year'], how='left')

        corr_map = df.loc[df['pow_above_base'] == 1].groupby(['site','season','day_of_week','time','year']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_st_pab')

        df = df.merge(corr_map, on=['site','season','day_of_week','time','year'], how='left')

        # median of var per site, month, time
        med_map = df.groupby(['site','month','time','year'])[var].median().rename(f'{var}_median_mt')
        df = df.merge(med_map, on=['site','month','time','year'], how='left')

        med_map = df.groupby(['site','season','time','year'])[var].median().rename(f'{var}_median_st')
        df = df.merge(med_map, on=['site','season','time','year'], how='left')

        # interaction term
        df[f'{var}_corr_dev_mt'] = df[f'{var}_power_corr_mt'] * (df[var] - df[f'{var}_median_mt'])
        df[f'{var}_corr_dev_st'] = df[f'{var}_power_corr_st'] * (df[var] - df[f'{var}_median_st'])

        # interaction term pab
        df[f'{var}_corr_dev_mt_pab'] = df[f'{var}_power_corr_mt_pab'] * (df[var] - df[f'{var}_median_mt'])
        df[f'{var}_corr_dev_st_pab'] = df[f'{var}_power_corr_st_pab'] * (df[var] - df[f'{var}_median_st'])

    df.fillna(0, inplace=True)

    for diff in [1,2,4,12]:
        df[f"temp_diff{diff if diff > 1 else ''}"] = df.groupby(['site'])['temp'].diff(diff).fillna(0)
        df[f"irr_diff{diff if diff > 1 else ''}"] = df.groupby(['site'])['irr'].diff(diff).fillna(0)

    features = [
        'temp', 'irr', 
        'temp_diff', 'irr_diff',
        'temp_diff2', 'irr_diff2',
        'temp_diff4', 'irr_diff4',
        'temp_diff12', 'irr_diff12'
    ]
    group_levels = [['site','time','year'], ['site','year']]
    suffixes = {
        ('site','time','year'): ('lag', 'peek'),
        ('site','year'): ('shift', 'pull')
    }
    shifts = [1, 2, 3, 4, 8, 12]

    for feature in features:
        for group in group_levels:
            suffix_pair = suffixes[tuple(group)]
            for direction, suffix in zip([1, -1], suffix_pair):
                for s in shifts:
                    col_name = f"{feature}_{suffix}{s if s > 1 else ''}"
                    df[col_name] = df.groupby(group)[feature].transform(lambda x, shift=s*direction: x.shift(shift))

    df['usage_lag'] = df.groupby(['site','time','year'])['power'].transform(lambda x: x.shift(1))
    df['usage_lag2'] = df.groupby(['site','time','year'])['power'].transform(lambda x: x.shift(2))
    df['usage_lag_dow'] = df.groupby(['site','day_of_week','time','year'])['power'].transform(lambda x: x.shift(1))
    df['usage_lag_dow2'] = df.groupby(['site','day_of_week','time','year'])['power'].transform(lambda x: x.shift(2))

    df['usage_peek'] = df.groupby(['site','time','year'])['power'].transform(lambda x: x.shift(-1))
    df['usage_peek2'] = df.groupby(['site','time','year'])['power'].transform(lambda x: x.shift(-2))
    df['usage_peek_dow'] = df.groupby(['site','day_of_week','time','year'])['power'].transform(lambda x: x.shift(-1))
    df['usage_peek_dow2'] = df.groupby(['site','day_of_week','time','year'])['power'].transform(lambda x: x.shift(-2))

    df['mean_usage_mdt'] = df.groupby(['site','month','day_of_week','time','year'])['power'].transform(lambda x: x.mean())
    df['mean_usage_sdt'] = df.groupby(['site','season','day_of_week','time','year'])['power'].transform(lambda x: x.mean())

    df['mean_usage_mdt_corr_dev_mt'] = (df['mean_usage_mdt'] + df['temp_corr_dev_mt']*1.5 + df['irr_corr_dev_mt'] * 0.05)
    df['mean_usage_sdt_corr_dev_st'] = (df['mean_usage_sdt'] + df['temp_corr_dev_st']*1.5 + df['irr_corr_dev_st'] * 0.05)

    df['mean_usage_mdt_corr_dev_mt_pab'] = (df['mean_usage_mdt'] + df['temp_corr_dev_mt_pab']*1.5 + df['irr_corr_dev_mt_pab'] * 0.05)
    df['mean_usage_sdt_corr_dev_st_pab'] = (df['mean_usage_sdt'] + df['temp_corr_dev_st_pab']*1.5 + df['irr_corr_dev_st_pab'] * 0.05)

    df['pow_monthly_max'] = df.groupby(['site','month','year'])['power'].transform('max')

    return df.fillna(0)

In [7]:
def create_day_response(data):
    df = data.copy()
    df['day_response'] = 0 # Becomes -1,0,1,2

    positive_flag_dates = df.groupby(["site", "date"]).filter(
        lambda df: (df["demand_response"] == 1).any() and not (df["demand_response"] == -1).any()
    )[["site", "date"]].drop_duplicates()

    negative_flag_dates = df.groupby(["site", "date"]).filter(
        lambda df: (df["demand_response"] == -1).any() and not (df["demand_response"] == 1).any()
    )[["site", "date"]].drop_duplicates()

    pos_and_neg_flag_dates = df.groupby(["site", "date"]).filter(
        lambda df: (df["demand_response"] == 1).any() and (df["demand_response"] == -1).any()
    )[["site", "date"]].drop_duplicates()

    # Create a tuple column for easier comparison
    df["site_date"] = list(zip(df["site"], df["date"]))

    # Create sets of tuples for comparison
    neg_set = set(zip(negative_flag_dates["site"], negative_flag_dates["date"]))
    pos_set = set(zip(positive_flag_dates["site"], positive_flag_dates["date"]))
    both_set = set(zip(pos_and_neg_flag_dates["site"], pos_and_neg_flag_dates["date"]))

    # Assign values
    df.loc[df["site_date"].isin(neg_set), "day_response"] = -1
    df.loc[df["site_date"].isin(pos_set), "day_response"] = 1
    df.loc[df["site_date"].isin(both_set), "day_response"] = 2

    # Optionally drop the helper column
    df.drop(columns="site_date", inplace=True)

    return df

In [8]:
prev_df = create_day_response(prev_df)
df = create_day_response(df)

In [9]:
print(len(prev_df.columns))
prev_df_wh = feature_engineering(prev_df)
print(len(prev_df_wh.columns))

19
307


In [10]:
print(len(df.columns))
df_wh = feature_engineering(df)
print(len(df_wh.columns))

19
307


# Split dataset

- df_no_response: Contains days with only zero demand response (predicted)
- df_response:    Contains days with any nonzero demand response (predicted)

In [11]:
df_no_response = pd.concat([df_wh[(df_wh['day_response'] == 0) & (df_wh['working_hours'] == 1)].copy(), prev_df_wh[(prev_df_wh['day_response'] == 0) & (prev_df_wh['working_hours'] == 1)].copy()]).sort_values(by=['site','timestamp']).reset_index(drop=True)
print(len(df_no_response)//(10*4))
df_no_response.head()

3127


,site,timestamp,temp,irr,power,demand_response,demand_response_capacity,busday,date,day_of_week,week,hour,time,quarter_hour,month,year,season,working_hours,day_response,baseline_pow,pow_above_base,site_on,li_flag,li_feature,temp_power_corr_mt,temp_power_corr_st,temp_power_corr_mt_pab,temp_power_corr_st_pab,temp_median_mt,temp_median_st,temp_corr_dev_mt,temp_corr_dev_st,temp_corr_dev_mt_pab,temp_corr_dev_st_pab,irr_power_corr_mt,irr_power_corr_st,irr_power_corr_mt_pab,irr_power_corr_st_pab,irr_median_mt,irr_median_st,irr_corr_dev_mt,irr_corr_dev_st,irr_corr_dev_mt_pab,irr_corr_dev_st_pab,temp_diff,irr_diff,temp_diff2,irr_diff2,temp_diff4,irr_diff4,...,temp_diff12_shift2,temp_diff12_shift3,temp_diff12_shift4,temp_diff12_shift8,temp_diff12_shift12,temp_diff12_pull,temp_diff12_pull2,temp_diff12_pull3,temp_diff12_pull4,temp_diff12_pull8,temp_diff12_pull12,irr_diff12_lag,irr_diff12_lag2,irr_diff12_lag3,irr_diff12_lag4,irr_diff12_lag8,irr_diff12_lag12,irr_diff12_peek,irr_diff12_peek2,irr_diff12_peek3,irr_diff12_peek4,irr_diff12_peek8,irr_diff12_peek12,irr_diff12_shift,irr_diff12_shift2,irr_diff12_shift3,irr_diff12_shift4,irr_diff12_shift8,irr_diff12_shift12,irr_diff12_pull,irr_diff12_pull2,irr_diff12_pull3,irr_diff12_pull4,irr_diff12_pull8,irr_diff12_pull12,usage_lag,usage_lag2,usage_lag_dow,usage_lag_dow2,usage_peek,usage_peek2,usage_peek_dow,usage_peek_dow2,mean_usage_mdt,mean_usage_sdt,mean_usage_mdt_corr_dev_mt,mean_usage_sdt_corr_dev_st,mean_usage_mdt_corr_dev_mt_pab,mean_usage_sdt_corr_dev_st_pab,pow_monthly_max
0,siteA,2019-01-01 10:00:00,27.5,817.0,15.04,0.0,0.0,1,2019-01-01,1,1,10,10:00:00,0,1,2019,0,1,0,5.76,1,7,1,17.940000,0.511101,0.555370,0.511101,-0.008398,25.90,25.050,0.817761,1.360657,0.817761,-0.020575,0.375639,0.184388,0.375639,0.383326,786.0,790.0,11.644821,4.978475,11.644821,10.349794,0.33,49.0,0.65,98.0,1.30,196.0,...,3.75,3.52,3.30,1.35,-0.60,0.94,0.78,0.62,4.65,3.20,3.70,0.0,0.0,0.0,0.0,0.0,0.0,637.0,637.0,638.0,638.0,640.0,642.0,630.0,624.0,617.0,611.0,400.0,181.0,620.0,604.0,587.0,572.0,448.0,283.0,0.0,0.0,0.0,0.0,21.18,14.76,5.05,5.51,8.010,5.209286,9.818883,7.499196,9.818883,5.695913,34.92
1,siteA,2019-01-01 10:15:00,24.6,856.0,15.31,0.0,0.0,1,2019-01-01,1,1,10,10:15:00,1,1,2019,0,1,0,5.76,1,8,1,17.541875,-0.748271,0.341621,-0.748271,-0.302010,25.98,24.950,1.032614,-0.119568,1.032614,0.105703,0.593603,0.084919,0.593603,0.175578,826.0,829.5,17.808096,2.250352,17.808096,4.652809,-2.90,39.0,-2.57,88.0,-1.92,186.0,...,3.97,3.75,3.52,1.84,-0.11,0.78,0.62,4.65,4.29,3.33,6.43,0.0,0.0,0.0,0.0,0.0,0.0,620.0,620.0,622.0,622.0,625.0,627.0,636.0,630.0,624.0,617.0,454.0,236.0,604.0,587.0,572.0,540.0,406.0,235.0,0.0,0.0,0.0,0.0,18.76,12.72,5.22,6.09,7.824,4.690714,10.263326,4.623881,10.263326,5.081910,34.92
2,siteA,2019-01-01 10:30:00,24.8,895.0,15.84,0.0,0.0,1,2019-01-01,1,1,10,10:30:00,2,1,2019,0,1,0,5.76,1,9,1,17.143750,-0.687966,0.344401,-0.687966,-0.228939,26.22,25.145,0.976912,-0.118818,0.976912,0.078984,0.579447,0.084023,0.579447,0.159917,866.0,869.5,16.803961,2.142583,16.803961,4.077872,0.20,39.0,-2.70,78.0,-2.05,176.0,...,4.20,3.97,3.75,2.32,0.37,0.62,4.65,4.29,3.93,3.44,6.07,0.0,0.0,0.0,0.0,0.0,0.0,604.0,604.0,605.0,605.0,609.0,612.0,620.0,636.0,630.0,624.0,506.0,291.0,587.0,572.0,540.0,509.0,365.0,188.0,0.0,0.0,0.0,0.0,18.24,11.72,4.40,6.18,7.632,4.691429,9.937566,4.620330,9.937566,5.013798,34.92
3,siteA,2019-01-01 10:45:00,25.0,933.0,16.23,0.0,0.0,1,2019-01-01,1,1,10,10:45:00,3,1,2019,0,1,0,5.76,1,10,1,16.745625,-0.670325,0.372860,-0.670325,-0.083015,26.38,25.195,0.925048,-0.072708,0.925048,0.016188,0.560297,0.020368,0.560297,0.037084,906.0,909.0,15.128022,0.488828,15.128022,0.890020,0.20,38.0,0.40,77.0,-2.17,165.0,...,0.94,4.20,3.97,2.81,0.86,4.65,4.29,3.93,3.57,3.58,5.71,0.0,0.0,0.0,0.0,0.0,0.0,587.0,587.0,589.0,590.0,593.0,597.0,604.0,620.0,636.0,630.0,559.0,346.0,572.0,540.0,509.0,478.0,324.0,141.0,0.0,0.0,0.0,0.0,17.55,11.72,4.95,6.13,7.988,4.717143,10.131973,4.632523,10.131973,4.785926,34.92
4,siteA,201

In [12]:
df_response = df_wh[(df_wh['day_response'] != 0) & (df_wh['working_hours'] == 1)].copy()
print(len(df_response)//(10*4))
df_response.head()

332


,site,timestamp,temp,irr,power,demand_response,demand_response_capacity,busday,date,day_of_week,week,hour,time,quarter_hour,month,year,season,working_hours,day_response,baseline_pow,pow_above_base,site_on,li_flag,li_feature,temp_power_corr_mt,temp_power_corr_st,temp_power_corr_mt_pab,temp_power_corr_st_pab,temp_median_mt,temp_median_st,temp_corr_dev_mt,temp_corr_dev_st,temp_corr_dev_mt_pab,temp_corr_dev_st_pab,irr_power_corr_mt,irr_power_corr_st,irr_power_corr_mt_pab,irr_power_corr_st_pab,irr_median_mt,irr_median_st,irr_corr_dev_mt,irr_corr_dev_st,irr_corr_dev_mt_pab,irr_corr_dev_st_pab,temp_diff,irr_diff,temp_diff2,irr_diff2,temp_diff4,irr_diff4,...,temp_diff12_shift2,temp_diff12_shift3,temp_diff12_shift4,temp_diff12_shift8,temp_diff12_shift12,temp_diff12_pull,temp_diff12_pull2,temp_diff12_pull3,temp_diff12_pull4,temp_diff12_pull8,temp_diff12_pull12,irr_diff12_lag,irr_diff12_lag2,irr_diff12_lag3,irr_diff12_lag4,irr_diff12_lag8,irr_diff12_lag12,irr_diff12_peek,irr_diff12_peek2,irr_diff12_peek3,irr_diff12_peek4,irr_diff12_peek8,irr_diff12_peek12,irr_diff12_shift,irr_diff12_shift2,irr_diff12_shift3,irr_diff12_shift4,irr_diff12_shift8,irr_diff12_shift12,irr_diff12_pull,irr_diff12_pull2,irr_diff12_pull3,irr_diff12_pull4,irr_diff12_pull8,irr_diff12_pull12,usage_lag,usage_lag2,usage_lag_dow,usage_lag_dow2,usage_peek,usage_peek2,usage_peek_dow,usage_peek_dow2,mean_usage_mdt,mean_usage_sdt,mean_usage_mdt_corr_dev_mt,mean_usage_sdt_corr_dev_st,mean_usage_mdt_corr_dev_mt_pab,mean_usage_sdt_corr_dev_st_pab,pow_monthly_max
1288,siteA,2020-01-14 10:00:00,25.10,791.0,19.26,0,0,1,2020-01-14,1,3,10,10:00:00,0,1,2020,0,1,2,2.4,1,9,1,17.790000,0.320467,0.339472,0.320467,0.339472,24.30,23.90,0.256374,0.407366,0.256374,0.407366,-0.304746,0.754250,-0.304746,0.754250,786.0,749.0,-1.523731,31.678517,-1.523731,31.678517,0.47,50.0,0.94,101.0,1.87,201.0,...,4.77,4.35,3.93,1.86,-0.20,5.20,4.79,4.38,3.99,2.37,0.00,642.0,642.0,641.0,640.0,639.0,637.0,643.0,643.0,644.0,644.0,646.0,647.0,630.0,616.0,603.0,590.0,367.0,148.0,628.0,613.0,598.0,583.0,464.0,301.0,24.62,25.62,23.20,0.0,6.41,29.70,23.55,23.26,22.3175,19.53875,22.625874,21.733725,22.625874,21.733725,48.29
1289,siteA,2020-01-14 10:15:00,25.16,831.0,23.52,1,0,1,2020-01-14,1,3,10,10:15:00,1,1,2020,0,1,2,2.4,1,10,1,17.265313,0.523750,0.412195,0.523750,0.412195,24.60,24.13,0.293300,0.424561,0.293300,0.424561,-0.684201,0.781687,-0.684201,0.781687,826.0,790.0,-3.421007,32.049182,-3.421007,32.049182,0.06,40.0,0.53,90.0,1.46,191.0,...,5.18,4.77,4.35,2.38,0.32,4.79,4.38,3.99,3.58,1.77,0.05,627.0,626.0,625.0,626.0,623.0,620.0,628.0,629.0,630.0,630.0,633.0,634.0,643.0,630.0,616.0,603.0,423.0,203.0,613.0,598.0,583.0,553.0,423.0,254.0,19.94,25.58,21.14,0.0,6.20,21.24,22.10,23.91,22.6675,20.06250,22.936400,22.301800,22.936400,22.301800,48.29
1290,siteA,2020-01-14 10:30:00,25.22,871.0,24.57,1,0,1,2020-01-14,1,3,10,10:30:00,2,1,2020,0,1,2,2.4,1,11,1,16.740625,-0.095051,-0.247950,-0.095051,-0.247950,24.90,24.30,-0.030416,-0.228114,-0.030416,-0.228114,-0.149589,0.397499,-0.149589,0.397499,866.0,832.0,-0.747945,15.502472,-0.747945,15.502472,0.06,40.0,0.12,80.0,1.06,181.0,...,5.60,5.18,4.77,2.91,0.83,4.38,3.99,3.58,3.17,1.19,0.10,612.0,612.0,611.0,610.0,607.0,604.0,613.0,614.0,615.0,616.0,618.0,621.0,628.0,643.0,630.0,616.0,478.0,258.0,598.0,583.0,553.0,524.0,383.0,206.0,25.09,21.46,20.30,0.0,6.74,25.55,20.18,22.65,21.9250,20.92250,21.841978,21.355452,21.841978,21.355452,48.29
1291,siteA,2020-01-14 10:45:00,25.28,910.0,27.06,1,0,1,2020-01-14,1,3,10,10:45:00,3,1,2020,0,1,2,2.4,1,12,1,16.215937,-0.099122,0.347562,-0.099122,0.347562,25.20,24.60,-0.007930,0.236342,-0.007930,0.236342,-0.140917,0.723665,-0.140917,0.723665,906.0,873.0,-0.563669,26.775616,-0.563669,26.775616,0.06,39.0,0.12,79.0,0.65,169.0,...,5.20,5.60,5.18,3.41,1.35,3.99,3.58,3.17,2.77,0.59,0.15,597.0,596.0,595.0,594.0,590.0,587.0,599.0,600.0,601.0,602.0,605.0,608.0,613.0,628.0,643.0,630.0,534.0,312.0,583.0,553.0,524.0,494.0,342.0,160.0,22.89,24.45,23.18,0.0,

# Regression

In [13]:
site_letters = [site[-1] for site in df['site'].unique()]
print(site_letters)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']


In [14]:
feats = ['day_of_week', 'month', 'li_feature', 'temp_lag', 'temp_lag3', 'temp_lag4', 'temp_peek', 'irr_lag', 'irr_lag2', 'irr_lag3', 'irr_lag4', 'irr_peek2', 'irr_peek3', 'irr_peek4', 'temp_shift4', 'irr_shift', 'irr_shift2', 'irr_shift3', 'irr_shift4', 'irr_pull', 'irr_pull2', 'irr_pull4', 'usage_lag_dow2', 'usage_lag_dow', 'hour', 'irr_pull3', 'temp_median_mt', 'irr_median_st', 'week', 'temp_pull3', 'irr_power_corr_st', 'irr_peek', 'baseline_pow', 'quarter_hour', 'temp_pull2', 'temp_lag2'] 
feats2 = ['irr_lag', 'irr_peek3', 'temp_lag', 'temp_peek2', 'season', 'month', 'day_of_week', 'hour', 'temp', 'irr', 'mean_usage_sdt_corr_dev_st', 'usage_lag', 'usage_lag_dow', 'usage_peek', 'usage_peek_dow', 'temp_shift3', 'temp_lag3', 'li_feature', 'baseline_pow', 'temp_pull3'] 
feats2_v2 = ['irr_lag', 'irr_peek3', 'temp_lag', 'temp_peek2', 'season', 'month', 'day_of_week', 'hour', 'temp', 'irr', 'mean_usage_sdt_corr_dev_st', 'usage_lag', 'usage_lag_dow', 'usage_peek', 'usage_peek_dow', 'temp_shift3', 'temp_lag3', 'li_feature', 'baseline_pow', 'temp_pull3', 'temp_shift12', 'irr_diff4_shift3']  # potentially better
feats3 = ['day_of_week', 'hour', 'month', 'mean_usage_sdt_corr_dev_st', 'li_feature', 'temp_lag', 'temp_shift4', 'week', 'irr_power_corr_st_pab', 'mean_usage_sdt', 'baseline_pow', 'usage_peek2', 'irr_median_mt', 'irr_power_corr_st', 'usage_lag', 'temp_shift', 'temp_power_corr_st', 'irr_shift4', 'temp_lag3', 'irr_pull2'] 
feats4 = ['day_of_week', 'hour', 'month', 'li_feature', 'temp_shift4', 'temp_lag', 'irr_diff2_peek2', 'week', 'irr_diff12_pull', 'temp_median_st', 'mean_usage_sdt_corr_dev_st_pab', 'temp_diff12_shift4', 'irr_diff4_pull3', 'usage_lag_dow', 'temp_lag2'] 

for site in site_letters:
    sub_no_response = df_no_response[df_no_response['site']==f'site{site}'].copy()

    X1 = sub_no_response[feats]
    X2 = sub_no_response[feats2]
    X2_v2 = sub_no_response[feats2_v2]
    X3 = sub_no_response[feats3]
    X4 = sub_no_response[feats4]
    y = sub_no_response['power']

    xgbr1 = xgb.XGBRegressor(random_state=42,eta=0.05778072586944159,subsample=0.6616312346972526,max_depth=9,colsample_bytree=0.6352137434228233, n_estimators=184)
    xgbr1.fit(X1, y)

    xgbr2 = xgb.XGBRegressor(random_state=42,eta=0.05513689802291822,subsample=0.5167731642508813,max_depth=9,colsample_bytree=0.7395754768661291, n_estimators=104)
    xgbr2.fit(X2, y)

    xgbr2_v2 = xgb.XGBRegressor(random_state=42,eta=0.05513689802291822,subsample=0.5167731642508813,max_depth=9,colsample_bytree=0.7395754768661291, n_estimators=104)
    xgbr2_v2.fit(X2_v2, y)

    xgbr3 = xgb.XGBRegressor(objective='reg:absoluteerror',random_state=42,eta=0.06449597248839985,subsample=0.9767027104796806,max_depth=8,colsample_bytree=0.450212514664335, n_estimators=197)
    xgbr3.fit(X3, y)

    xgbr4 = xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.0636975789822504,subsample=0.6608736639449571,max_depth=9,colsample_bytree=0.6033560282663871, n_estimators=162)
    xgbr4.fit(X4, y)


    X1_test = df_response.loc[df_response['site']==f'site{site}', feats]
    X2_test = df_response.loc[df_response['site']==f'site{site}', feats2]
    X2_test_v2 = df_response.loc[df_response['site']==f'site{site}', feats2_v2]
    X3_test = df_response.loc[df_response['site']==f'site{site}', feats3]
    X4_test = df_response.loc[df_response['site']==f'site{site}', feats4]
    df_response.loc[df_response['site']==f'site{site}', 'power_prediction_xgbr'] = xgbr1.predict(X1_test)
    df_response.loc[df_response['site']==f'site{site}', 'power_prediction_xgbr2'] = xgbr2.predict(X2_test)
    df_response.loc[df_response['site']==f'site{site}', 'power_prediction_xgbr2_v2'] = xgbr2_v2.predict(X2_test_v2)
    df_response.loc[df_response['site']==f'site{site}', 'power_prediction_xgbr3'] = xgbr3.predict(X3_test)
    df_response.loc[df_response['site']==f'site{site}', 'power_prediction_xgbr4'] = xgbr4.predict(X4_test)


df_response['capacity_prediction'] = 0.0
df_response['capacity_prediction_xgbr'] = 0.0
df_response['capacity_prediction_xgbr2'] = 0.0
df_response['capacity_prediction_xgbr2_v2'] = 0.0
df_response['capacity_prediction_xgbr3'] = 0.0
df_response['capacity_prediction_xgbr4'] = 0.0

df_response.loc[df_response['demand_response'] != 0, 'capacity_prediction_xgbr'] = df_response['power'] - (df_response['power_prediction_xgbr'])
df_response.loc[df_response['demand_response'] != 0, 'capacity_prediction_xgbr2'] = df_response['power'] - (df_response['power_prediction_xgbr2'])
df_response.loc[df_response['demand_response'] != 0, 'capacity_prediction_xgbr2_v2'] = df_response['power'] - (df_response['power_prediction_xgbr2_v2'])
df_response.loc[df_response['demand_response'] != 0, 'capacity_prediction_xgbr3'] = df_response['power'] - (df_response['power_prediction_xgbr3'])
df_response.loc[df_response['demand_response'] != 0, 'capacity_prediction_xgbr4'] = df_response['power'] - (df_response['power_prediction_xgbr4'])

In [15]:
df_response['capacity_prediction'] = df_response['capacity_prediction_xgbr'] * 0.55 + df_response['capacity_prediction_xgbr2'] * 0.25 + df_response['capacity_prediction_xgbr3'] * 0.2
df_response['capacity_prediction_v2'] = df_response['capacity_prediction_xgbr'] * 0.45 + df_response['capacity_prediction_xgbr2_v2'] * 0.3 + df_response['capacity_prediction_xgbr3'] * 0.15 + df_response['capacity_prediction_xgbr4'] * 0.1 # potentially better

- Join preds to df

In [16]:
merged = pd.merge(df, df_response[['site','timestamp','capacity_prediction','capacity_prediction_v2']], how='left', on=['site', 'timestamp']).fillna(0).sort_values(['site', 'date', 'timestamp'])
merged

,site,timestamp,temp,irr,power,demand_response,demand_response_capacity,busday,date,day_of_week,week,hour,time,quarter_hour,month,year,season,working_hours,day_response,capacity_prediction,capacity_prediction_v2
0,siteA,2020-01-01 00:00:00,19.89,0.0,12.00,0,0,1,2020-01-01,2,1,0,00:00:00,0,1,2020,0,0,0,0.0,0.0
1,siteA,2020-01-01 00:15:00,19.79,0.0,12.00,0,0,1,2020-01-01,2,1,0,00:15:00,1,1,2020,0,0,0,0.0,0.0
2,siteA,2020-01-01 00:30:00,19.70,0.0,12.00,0,0,1,2020-01-01,2,1,0,00:30:00,2,1,2020,0,0,0,0.0,0.0
3,siteA,2020-01-01 00:45:00,19.60,0.0,12.00,0,0,1,2020-01-01,2,1,0,00:45:00,3,1,2020,0,0,0,0.0,0.0
4,siteA,2020-01-01 01:00:00,19.50,0.0,12.00,0,0,1,2020-01-01,2,1,1,01:00:00,0,1,2020,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289435,siteM,2020-08-31 22:45:00,13.47,0.0,56.33,0,0,1,2020-08-31,0,36,22,22:45:00,3,8,2020,2,0,0,0.0,0.0
289436,siteM,2020-08-31 23:00:00,13.30,0.0,56.33,0,0,1,2020-08-31,0,36,23,23:00:00,0,8,2020,2,0,0,0.0,0.0
289437,siteM,2020-08-31 23:15:00,13.35,0.0,56.33,0,0,1,2020-08-31,0,36,23,23:15:00,1,8,2020,2,0,0,0.0,0.0
289438,siteM,2020-08-31 23:30:00,13.40,0.0,56.33,0,0,1,2020-08-31,0,36,23,23:30:00,2,8,2020,2,0,0,0.0,0.0


# Clip Predictions

In [17]:
small_site_neg_clips = (-.29, .225) # percentages of site's max value to clip demand_response_capacity to
small_site_pos_clips = (0.0, .425)
large_site_neg_clips = (-.25, .125)
large_site_pos_clips = (0.0, .3)
pred_cols = [col for col in merged.columns if 'capacity_prediction' in col and 'xgb' not in col]
print(pred_cols)
for site in site_letters:
    site_label = f'site{site}'
    years = sorted(merged.loc[merged['site'] == site_label]['year'].unique())
    for year in years:
        for col in pred_cols:
            max_pow = merged[(merged['site'] == site_label) & (merged['working_hours'] == 1) & (merged['year'] == year)]['power'].max()
            if max_pow < 100:
                # neg clips
                merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == -1) & (merged['year'] == year), f'{col}_clipped'] = np.clip(merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == -1) & (merged['year'] == year)][col], small_site_neg_clips[0]*max_pow, small_site_neg_clips[1]*max_pow)
                # pos clips
                merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == 1) & (merged['year'] == year), f'{col}_clipped'] = np.clip(merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == 1) & (merged['year'] == year)][col], small_site_pos_clips[0]*max_pow, small_site_pos_clips[1]*max_pow)
            else:
                # neg clips
                merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == -1) & (merged['year'] == year), f'{col}_clipped'] = np.clip(merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == -1) & (merged['year'] == year)][col], large_site_neg_clips[0]*max_pow, large_site_neg_clips[1]*max_pow)
                # pos clips
                merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == 1) & (merged['year'] == year), f'{col}_clipped'] = np.clip(merged.loc[(merged['site'] == site_label) & (merged['demand_response'] == 1) & (merged['year'] == year)][col], large_site_pos_clips[0]*max_pow, large_site_pos_clips[1]*max_pow)
merged.fillna(0, inplace=True)

['capacity_prediction', 'capacity_prediction_v2']


# Join to Submission

In [18]:
capacity_prediction_col_name = 'capacity_prediction_clipped'

submission = pd.read_csv("./data/test_data_v3.csv")
col_names = list(submission.columns[:2]) + ['Demand_Response_Flag','Demand_Response_Capacity_kW']
submission['Timestamp_Local'] = pd.to_datetime(submission['Timestamp_Local'])
submission['Demand_Response_Flag'] = merged['demand_response']
submission = pd.merge(submission, merged[['site', 'timestamp', capacity_prediction_col_name]], how='left', left_on=['Site', 'Timestamp_Local'], right_on=['site', 'timestamp'])
submission = submission[['Site', 'Timestamp_Local', 'Demand_Response_Flag', capacity_prediction_col_name]].fillna(0)
submission.columns = col_names
submission

,Site,Timestamp_Local,Demand_Response_Flag,Demand_Response_Capacity_kW
0,siteA,2020-01-01 00:00:00,0,0.0
1,siteA,2020-01-01 00:15:00,0,0.0
2,siteA,2020-01-01 00:30:00,0,0.0
3,siteA,2020-01-01 00:45:00,0,0.0
4,siteA,2020-01-01 01:00:00,0,0.0
...,...,...,...,...
289435,siteM,2020-08-31 22:45:00,0,0.0
289436,siteM,2020-08-31 23:00:00,0,0.0
289437,siteM,2020-08-31 23:15:00,0,0.0
289438,siteM,2020-08-31 23:30:00,0,0.0


In [19]:
submission.to_csv("./data/submission.csv")